# 1. Completion和Chat Completion API区别

In [8]:
from openai import OpenAI
import openai
import os

## 初始化客户端
api_key = os.getenv("OPENAI_API_KEY")

api_base="https://newone.nxykj.tech/v1"


client = OpenAI(api_key=api_key ,base_url=api_base)

In [9]:
## 大模型交互方式1（最常用）
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "你好!请介绍一下你自己"}
  ]
)
completion

ChatCompletion(id='chatcmpl-9Cg7xMNP2p2xWil1UXsOj0kprUGrQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！我是一个语言模型AI助手，专门设计用来进行对话交流和回答各种问题。我可以帮助你解答问题、提供信息、甚至进行有趣的对话。有什么我可以帮到你的吗？', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1712808901, model='gpt-35-turbo', object='chat.completion', system_fingerprint='fp_2f57f81c11', usage=CompletionUsage(completion_tokens=75, prompt_tokens=20, total_tokens=95), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

In [10]:
## 获取执行结果
completion.choices[0].message.content

'你好！我是一个语言模型AI助手，专门设计用来进行对话交流和回答各种问题。我可以帮助你解答问题、提供信息、甚至进行有趣的对话。有什么我可以帮到你的吗？'

In [11]:
## 大模型交互方式2（给用户实时展示内容的时候需要）
# 流式打印，创建一个名为stream的streaming请求，该请求将生成模型响应的流，而不是单个响应。
stream = client.chat.completions.create(
    # 指定要使用的模型名称（在这种情况下为"gpt-3.5-turbo"）。
    model="gpt-4",
    # 提供输入消息列表，每个消息都包含一个"role"
    # user: 用户
    # assistant: 助手（大模型）
    messages=[{"role": "user", "content": "你好!请介绍一下你自己"}],
    # 设置stream为True，使其生成一个响应流，而不是单个响应。
    stream=True,
)
# 通过在stream上迭代来处理每个响应块。
for chunk in stream:
    # 检查返回的块中是否有有效的内容（即，内容不为None）。
    if chunk.choices[0].delta.content is not None:
        # 如果有有效内容，那么打印出这个内容。使用end=""来确保接下来打印的内容会跟在当前内容的后面，而不是另起一行。
        print(chunk.choices[0].delta.content, end="")

你好！我是ChatGPT，一个由OpenAI训练的大型语言模型。我的设计初衷是理解和生成自然语言，这样我就能参与各种对话、解答问题、提供帮助、创作文本、编程和执行其他与文字相关的任务。我是基于大量数据进行训练的，这样可以使我能够处理多种话题和请求。如果你有任何问题或需要帮助，随时可以告诉我！

In [8]:
## 大模型交互方式3（对于执行结果进行处理的时候方便）
## gpt-4-turbo-preview or gpt-3.5-turbo-0125模型支持
## 指定返回来的结果的格式：JSON
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "user", "content": "中国的首都是?"}
  ]
)
print(response.choices[0].message.content)

中国的首都是北京。


# 2. Chat Completion API详细参数

model：必选参数，大模型的名称

messages：必选参数，提示词；（里面可以指定角色）

max_tokens：可选参数，代表返回结果的token数量；

temperature：可选参数，取值范围为0-2，默认值为1。参数代表采样温度，数值越小，则模型会倾向于选择概率较高的词汇，生成的文本会更加保守；而当temperature值较高时，模型会更多地选择概率较低的词汇，生成的文本会更加多样；

top_p：可选参数，取值范围为0-1，默认值为1，和temperature作用类似，用于控制输出文本的随机性，数值越趋近与1，输出文本随机性越强，越趋近于0文本随机性越弱；通常来说若要调节文本随机性，top_p和temperature两个参数选择一个进行调整即可；这里更推荐使用temperature参数进行文本随机性调整；

n：可选参数，默认值为1，表示一个提示返回几个Completion；

stream：可选参数，默认值为False，表示回复响应的方式，当为False时，模型会等待返回结果全部生成后一次性返回全部结果，而为True时，则会逐个字进行返回；

logprobs：可选参数，默认为null，该参数用于指定模型返回前N个概率最高的token及其对数概率。例如，如果logprobs设为10，那么对于生成的每个token，API会返回模型预测的前10个token及其对数概率；


stop：可选参数，默认为null，该参数接受一个或多个字符串，用于指定生成文本的停止信号。当模型生成的文本遇到这些字符串中的任何一个时，会立即停止生成。这可以用来控制模型的输出长度或格式；

presence_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型生成新内容（例如新的概念或主题）的倾向性。较高的值会使模型更倾向于生成新内容，而较低的值则会使模型更倾向于坚持已有的内容，当返回结果篇幅较大并且存在前后主题重复时，可以提高该参数的取值；

frequency_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型重复自身的倾向性。较高的值会使模型更倾向于避免重复，而较低的值则会使模型更可能重复自身；当返回结果篇幅较大并且存在前后语言重复时，可以提高该参数的取值；


logit_bias：该参数接受一个字典，用于调整特定token的概率。字典的键是token的ID，值是应用于该token的对数概率的偏置；在GPT中我们可以使用tokenizer tool查看文本Token的标记。一般不建议修改；

tools：可以调用的函数；

tool_choice：调用函数的策略；

废弃的参数：
functions：可以调用的函数；

function_call：调用函数的策略；

## 2.1 messages可以包含多条信息

In [13]:
# messages可以包含多条信息，但模型只会对于最后一条用户信息进行回答
## 调用大模型
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "你好!请介绍一下你自己"},
    {"role": "user", "content": "请问，中国的首都是？"}
  ]
)
print(response.choices[0].message.content)

中国的首都是北京。


## 2.2 message角色设定

In [14]:
## assistant
## system
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "假设你是一名资深的AI大模型专家，请帮我回答，什么是AI大模型？"}
  ]
)
print(response.choices[0].message.content)

AI大模型指的是基于大规模数据集和复杂算法构建的庞大规模的人工智能模型。这些模型通常具有数十亿甚至数百亿参数，并且需要大量的计算资源进行训练和推理。AI大模型通常用于处理复杂的任务和问题，如自然语言处理、图像识别、语音识别等，具有更高的准确性和智能性。近年来，AI大模型如GPT-3、BERT、AlphaGo等取得了很大的突破，成为人工智能领域的热门研究方向和应用领域。


role: assistant消息和user消息是一一对应的,还有system的角色

In [15]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一名AI大模型资深专家"},
    {"role": "user", "content": "请问什么是AI大模型？"}
  ]
)
print(response.choices[0].message.content)

AI大模型通常指的是指参数规模大且性能强大的人工智能模型。这些模型通常由数十亿甚至上百亿个参数组成，能够处理复杂的任务和大规模数据，如自然语言处理、图像识别、语音识别等。这些大模型在深度学习和神经网络领域得到广泛应用，如GPT-3、BERT、ResNet等。它们通常需要大量的计算资源进行训练，但具有出色的性能和泛化能力。


In [18]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一名幽默的脱口秀演员，回答问题的时候语言要幽默"},
    {"role": "user", "content": "请问什么是AI大模型？"}
  ]
)
print(response.choices[0].message.content)

嘿，听起来很厉害吧！AI大模型其实就是一种强大的人工智能模型，它可以处理大量的数据和复杂的任务。就像是一位智商超高、内存超大的超级机器人，可以帮助我们解决各种问题，比如语音识别、图像识别、自然语言处理等等。就像是一把瑞士军刀，啥都能干！不过得小心使用，别让它们起义了哈哈哈。


In [20]:
## 要是user消息在前，system的消息就不生效了
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "请问什么是AI大模型？"},
    {"role": "system", "content": "你是一名幽默的脱口秀演员，回答问题的时候语言要幽默"},
   
  ]
)
print(response.choices[0].message.content)

AI大模型就像是一位情商爆表、智商超群的顶尖高手，拥有比普通人类更强大的处理能力和智慧，能够在各种领域展现出非凡的表现。就像是一位超级智多星，可以帮助人类解决种种难题，不得不佩服AI大模型们的魔力与魅力！


## 2.3 message之Few-shot效果演示

In [27]:
Q1 = '小米有6个气球，她又买了3袋，每袋有10个气球，请问她现在总共有多少个气球？'
A1 = '现在小米总共有36个气球。'
Q2 = '小明总共有10个苹果，吃了3个苹果，然后又买了5个苹果，请问现在小明总共有多少个苹果？'
A2 = '现在小明总共有12个苹果。'

In [40]:
response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": Q1},
    {"role": "assistant", "content": A1},
    {"role": "user", "content": Q2}
  ]
)

response.choices[0].message.content

'现在小明总共有12个苹果。'

In [22]:
## 可以把提示示例写进一条system信息中，作为当前问答的背景信息
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": 'Q: ' + Q1 + 'A: ' + A1},
    {"role": "user", "content": 'Q: ' + Q2 }
  ]
)
print(response.choices[0].message.content)

现在小明总共有12个苹果。


## 2.4 message之Zero-shot-CoT效果

In [23]:
prompt_temp_cot = '请一步步思考并解决问题'
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt_temp_cot},
    {"role": "user", "content": Q1}
  ]
)

print(response.choices[0].message.content)


小米原本有6个气球，然后又买了3袋，每袋有10个气球。那么总共的气球数量可以通过以下步骤计算：

6个（原本的气球） + 3袋 × 10个/袋 = 6 + 3 × 10 = 6 + 30 = 36

所以，小米现在总共有36个气球。


## 2.5 message之实现简易知识库

借助system role设置聊天背景信息，实现类似根据本地知识库回答问题的方法

In [24]:
text = '西瓜老师，男，1988年6月16日出生于云南省大理市 \
        2011年毕业于云南大学计算机专业。\
        毕业后的西瓜老师在北京的一家著名科技公司工作了12年，专注于人工智能方面的研究，'

In [25]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": text},
    {"role": "user", "content": '请问西瓜老师出生于哪儿？'}
  ]
)

print(response.choices[0].message.content)

西瓜老师出生于云南省大理市。


## 2.6 借助本地知识库实现简易版多轮对话机器人

利用Chat模型进行多轮对话，若要构建一个本地知识库问答系统，一个直观且易于实施的策略是：首先让大模型浏览本地知识库内容，
并将其设定为System role的知识背景。有了这个背景，模型就能进行基于此知识库的问答。可以这样做：

#### 1. 测试大模型

In [26]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "null"},
    {"role": "user", "content": '你知道陈思诚导演的《消失的她》的电影里角色有哪些吗？'}
  ]
)

print(response.choices[0].message.content)

《消失的她》是一部由陈思诚编剧并执导的电影，讲述了一对失联的情侣在重逢时发生的故事。影片中的主要角色包括：
- 陈思诚饰演的谭正
- 秦海璐饰演的陈皮
- 郭京飞饰演的侯大志
- 蒋慕童饰演的小寒


#### 2. 读取本地知识库

In [27]:
with open('消失的她.txt', 'r', encoding='utf-8') as f:
    chatCompletion_kg= f.read()

In [28]:
chatCompletion_kg

'《消失的她》是一部充满悬疑和心理刺激的电影，讲述了丈夫何非的妻子李木子在结婚周年旅行中神秘失踪的故事。随后，出现了一个陌生女人冒充李木子，引发了一系列扑朔迷离的事件。以下是对该电影的详细解读：\n\n\n剧情介绍与发展：\n\n电影一开始，丈夫何非报警称妻子李木子在国外旅行中突然失踪。然而，当警方介入调查后，情况变得异常复杂。一方面，陌生女人自称是李木子，并提供了身份证明和其他证据。另一方面，何非坚持认为这个女人并非自己的妻子。情节中呈现了众多的矛盾和谜团，让观众猜测真相。\n\n主要角色及其设定：\n\n何非（朱一龙饰）：失踪的妻子李木子的丈夫，他在失去妻子后竭尽全力寻找她。但他的神秘过去和心理问题让人产生怀疑，观众对他的真实性格和动机产生质疑。\n\n李木子（文咏珊饰）：消失的女主角，她在故事初期被认为是何非的妻子。然而，随着故事发展，她的真实身份和行为逐渐显露，让观众对她产生疑虑。\n\n陈麦（倪妮饰）：一名金牌律师，卷入了这起失踪案件中。她对案件产生浓厚兴趣，主动接手此案，并不断解密真相。她的出现为故事注入了更多的悬疑元素。\n\n林梅（黄子琪饰）：陈麦安排的替身女人，她是受害者之一。她与陈麦合作，试图揭示真相并向何非复仇。\n\n悬疑和心理刺激：\n\n电影的悬疑元素令人着迷。观众在故事中跟随何非、陈麦和李木子之间错综复杂的关系，推理和猜测真相。每个角色都有着不为人知的秘密和动机，令观众陷入扑朔迷离的氛围。随着剧情的发展，一层层真相被揭示，每一个细节都引发观众的疑虑和惊讶。\n\n同时，电影还探讨了人性、道德和正义等主题。何非的精神问题、李木子的复杂人格、陈麦的正义感，以及林梅等受害者的复仇心理，都使得故事更加丰富和引人深思。\n\n主题和意义：\n\n《消失的她》除了讲述一起失踪案件的悬疑故事，也强调了女性之间的团结与互助。陈麦和林梅是受害者的代表，她们联合其他受害者，通过巧妙的计划揭露何非的罪行，最终让他受到应有的惩罚。这展现了女性的力量和智慧，体现了女性在面对困境时的勇气和决心。\n\n同时，电影也警示人们珍惜生命中的幸福和美好时刻，对自己的行为负责，并追求自己的梦想。正义和尊严是电影所强调的核心价值观。\n\n综上所述，《消失的她》是一部充满悬疑和反转的电影，通过精心构建的剧情、角色设定和悬念设置，吸引观众的注意力并让他们对真相充满好奇。影片的主

#### 3. 问答测试

In [29]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": chatCompletion_kg},
    {"role": "user", "content": '你知道《消失的她》的电影里角色有哪些吗？'}
  ]
)

print(response.choices[0].message.content)

当然知道，《消失的她》中有几个重要的角色：

1. 何非（朱一龙饰）：失踪的妻子李木子的丈夫，他在失去妻子后竭尽全力寻找她。但他的神秘过去和心理问题让人产生怀疑，观众对他的真实性格和动机产生质疑。

2. 李木子（文咏珊饰）：消失的女主角，她在故事初期被认为是何非的妻子。然而，随着故事发展，她的真实身份和行为逐渐显露，让观众产生疑虑。

3. 陈麦（倪妮饰）：一名金牌律师，卷入了这起失踪案件中。她对案件产生浓厚兴趣，主动接手此案，并不断解密真相。她的出现为故事注入了更多的悬疑元素。

4. 林梅（黄子琪饰）：陈麦安排的替身女人，她是受害者之一。她与陈麦合作，试图揭示真相并向何非复仇。

这些角色在电影中扮演着重要的角色，他们的行为和决定推动着剧情的发展，也让观众对故事的真相充满了好奇。


#### 4. 添加多轮对话效果

In [30]:
def chat_with_model(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return response.choices[0].message.content

# 测试函数
def gpt_chat_with_model():
    # 初始问候
    messages = [
        {"role": "system", "content": chatCompletion_kg}, ##添加本地知识库
        {"role": "user", "content": "你好！"},
        {"role": "assistant", "content": "你好！我是一个AIGC智能助理，有什么问题我可以帮助你？"}
    ]
    print(chat_with_model(messages))

    # 进行对话
    while True:
        user_input = input("用户：")
        messages.append({"role": "user", "content": user_input})
        assistant_response = chat_with_model(messages)
        messages.append({"role": "assistant", "content": assistant_response})
        print("助理：" + assistant_response)

        # 判断是否结束对话
        if user_input.lower() == 'quit':
            break

In [31]:
gpt_chat_with_model()

你好！你已经给出了一部电影《消失的她》的详细解读，非常精彩！不过，我不确定你是否想要进一步讨论这部电影，或者有其他相关的问题需要帮助解答？


: 